In [1]:
import pandas as pd
df = pd.read_csv('ssbpts.csv')

df2 = df.query('MidLat > 30 and MidLat < 75 and MidLon < -60 and MidLon > -135 ')

try:
    dfe = pd.read_csv('errors.csv', index_col=0)
    df3 = pd.concat([df2, dfe], axis = 1)
    df4 = df3.query('Status == "OK"')
except FileNotFoundError:
    dfe = pd.DataFrame()
    df4 = df2

In [2]:
import obspy
from obspy.signal.filter import envelope
from scipy.signal import hilbert

from numpy import argmax, zeros

filename = 'data/mseed/%012d.mseed' % 1031038

rbin = zeros(200)

snr ={}

for row in df4.iterrows():
    bpindex = row[0]
    filename = 'data/mseed/%012d.mseed' % bpindex
    st = obspy.read(filename).detrend().taper(0.05).filter('lowpass',freq=0.1,corners=4)
    
    tr = st[0]
    
    noise   = tr.copy().trim(tr.stats.starttime, tr.stats.starttime+100)
    signal  = tr.copy().trim(tr.stats.starttime+100, tr.stats.starttime+200)
    
    if len(noise.data) == 0 or len(signal.data) == 0:
        snr[bpindex] = -1.0
        continue
    
    snr[bpindex]    = max(abs(envelope(signal.data))) / max(abs(envelope(noise.data)))
    

    

/Users/mancinelli/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:27: RuntimeWarning: invalid value encountered in double_scalars
/Users/mancinelli/anaconda/lib/python3.6/site-packages/obspy/signal/detrend.py:35: RuntimeWarning: invalid value encountered in true_divide
  data -= x1 + np.arange(ndat) * (x2 - x1) / float(ndat - 1)


In [3]:
snr.items()
dfout = pd.DataFrame(snr, index=["SNR"]).T

In [4]:
dfout.to_csv('snr.csv')

In [5]:
dfout.query("SNR > 3.0")

,SNR
2301,3.074457
4581,3.286390
5709,4.271263
11379,3.588503
11382,3.897003
11397,6.295181
11424,3.594863
12116,3.742650
12855,3.693870
31268,3.120215
